In [47]:
""" Monster Database Interface """
from os import getenv
import pandas as pd
from certifi import where
from dotenv import load_dotenv
from MonsterLab import Monster
from pandas import DataFrame
from pymongo import MongoClient
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from data import MongoDB


In [50]:
db = MongoDB("Collection")
db.reset()
print(db.count())

db.seed(amount=2500)
print(db.count())

df = db.dataframe()

if isinstance(df, pd.DataFrame):
    print("df is a Pandas DataFrame")
else:
    print("df is not a Pandas DataFrame")
print(df.head())

Collection 'Collection' has been reset.
0
There were 2500 documents inserted.
2500
df is a Pandas DataFrame
                        _id             Name       Type  Level  Rarity  \
0  65d7a2c240fc809904f0b436    Goblin Knight   Devilkin      5  Rank 0   
1  65d7a2c240fc809904f0b437     Magma Spirit        Fey     17  Rank 1   
2  65d7a2c240fc809904f0b438  Steam Elemental  Elemental      4  Rank 2   
3  65d7a2c240fc809904f0b439         Revenant     Undead      3  Rank 0   
4  65d7a2c240fc809904f0b43a      Dust Mephit  Elemental     15  Rank 1   

   Damage  Health  Energy  Sanity            Timestamp  
0   5d2+2    9.18    9.41   10.94  2024-02-22 11:38:42  
1  17d4+1   67.77   67.71   66.57  2024-02-22 11:38:42  
2     4d6   25.29   23.60   25.40  2024-02-22 11:38:42  
3   3d2+4    5.22    5.86    6.71  2024-02-22 11:38:42  
4  15d4+1   60.28   61.54   60.24  2024-02-22 11:38:42  


In [41]:
import altair as alt
from altair import Chart, Tooltip
import pandas as pd

alt.data_transformers.enable('vegafusion')

def chart(df: pd.DataFrame, x: str, y:  str, target: str) -> alt.Chart:
    """ Create a chart function that takes a dataframe and a target column and returns a chart """
    
    graph = alt.Chart(df).mark_circle(size=100).encode(
        x=alt.X(x, axis=alt.Axis(labelColor='#AAAAAA', titleColor='#AAAAAA', gridColor="#333333")),
        y=alt.Y(y, axis=alt.Axis(labelColor='#AAAAAA', titleColor='#AAAAAA', gridColor='#333333')),
        color=alt.Color(target, legend=alt.Legend(labelColor='#AAAAAA', titleColor='#AAAAAA')),
        tooltip=list(df.columns)
    ).properties(
        title=alt.TitleParams(text=f"{y} by {x} for {target}", color='#AAAAAA', fontSize=23),
        width=450,
        height=500,
        padding={"left": 33, "right": 33, "top": 50, "bottom": 33},
        background="#232323"
    )
    return graph


In [43]:
chart(db, 'Level', 'Energy', 'Rarity')


AttributeError: 'MongoDB' object has no attribute 'columns'

In [28]:
from pandas import DataFrame
from altair import Chart, Tooltip


def chart(df: DataFrame, x: str, y: str, target: str) -> Chart:

    properties = {
        'width': 600,    # Set the desired width of the chart
        'height': 400,   # Set the desired height of the chart
        'background': '#1E1E1E',  # Dark background color to fit Bandersnatch theme
        'padding': {'left': 20, 'right': 20, 'top': 10, 'bottom': 10}  # Adjust padding as needed
    }
    graph = Chart(
        df,
        title=f"{y} by {x} for {target}"
    ).mark_circle(size=100).encode(
        x=x,
        y=y,
        color=target,
        tooltip=Tooltip(df.columns.to_list())
    ).configure_title(fontSize=20, color='#FFFFFF')

    return graph

In [15]:
df.head()

,Name,Type,Level,Rarity,Damage,Health,Energy,Sanity,Timestamp
0,Pit Lord,Devilkin,14,Rank 1,14d4+2,54.57,57.59,55.12,2024-02-15 16:49:39
1,Diamond Archfey,Fey,10,Rank 1,10d4+2,38.25,39.21,39.54,2024-02-15 16:49:39
2,Shadow Elemental,Elemental,1,Rank 0,1d2+4,1.61,1.82,1.73,2024-02-15 16:49:39
3,Hell Hound,Demonic,11,Rank 2,11d6,65.03,67.99,65.18,2024-02-15 16:49:39
4,Bronze Faerie,Fey,3,Rank 4,3d10+1,32.69,30.54,30.11,2024-02-15 16:49:39


In [28]:
X = df[['Level', 'Energy', 'Health', 'Sanity']]
y = df['Rarity']

In [29]:
print(X.shape)
print(y.shape)

(2500, 4)
(2500,)


In [30]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [31]:
X_train.shape

(2000, 4)

In [32]:
X_val.shape

(500, 4)

In [51]:
from sklearn.preprocessing import OrdinalEncoder
model_rf = RandomForestClassifier(random_state=42, n_jobs=-1, max_depth=7)
model_rf.fit(X_train, y_train)

print('Forest: Training Accuracy:', model_rf.score(X_train, y_train))
print('Forest: Validation Accuracy', model_rf.score(X_val, y_val))

Forest: Training Accuracy: 0.975
Forest: Validation Accuracy 0.944


In [60]:
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

y_train_encoded = label_encoder.fit_transform(y_train)
y_val_encoded = label_encoder.transform(y_val)

xg_classifier = XGBClassifier(n_estimators=10, max_depth=7, random_state=42, eval_metric='merror')
xg_classifier.fit(X_train,y_train_encoded)

print('Validation Accuracy: XGBoost', xg_classifier.score(X_train, y_train_encoded))
print('Validation Accuracy: XGBoost', xg_classifier.score(X_val, y_val_encoded))

Validation Accuracy: XGBoost 0.9945
Validation Accuracy: XGBoost 0.958


In [45]:
from sklearn.ensemble import AdaBoostClassifier

ada_classifier = AdaBoostClassifier(n_estimators=50, learning_rate=2, random_state=42)
ada_classifier.fit(X_train,y_train)


print('Training Accuracy: Adaboost', ada_classifier.score(X_train, y_train))
print('Validation Accuracy: Adaboost', ada_classifier.score(X_val, y_val))

Training Accuracy: Adaboost 0.5065
Validation Accuracy: Adaboost 0.502
